In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" # select which GPU(s) to use

In [2]:
import numpy as np
import math
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from sklearn.utils import class_weight

import itertools
print(tf.__version__)

from platform import python_version
print(python_version())

2.4.0
3.8.8


In [3]:
# Setting the dtype policy for tensor cores
mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 3060, compute capability 8.6


In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    assert tf.config.experimental.get_memory_growth(physical_devices[0])
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

In [5]:
train_dir = './datasets/train/'
validate_dir = './datasets/validate/'
datasample_period = 300
prediction_period = 30
feature_columns = 40
band_size = 0.001
capacity_const = 6 * 40 * 65000 # calibration depends on single test run
n_train_files = len(os.listdir(train_dir))
n_validate_files = len(os.listdir(validate_dir))
steps_per_epoch = 0 # to be determined from generator

In [6]:
# Prepare a directory to store all the checkpoints.
checkpoint_dir = './models/checkpoint'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [26]:
files = []
for subdir, dirs, files in os.walk(validate_dir):
    files=files
all_arrays = []
for npfile in files:
    all_arrays.append(np.load(os.path.join(validate_dir, npfile)))
np.save('binance_dateset_all.npy', np.concatenate(all_arrays))

In [8]:
train_data = np.load('./binance_dateset_all.npy')

In [9]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data, sample_size, prediction_period, feature_num, band_size, batch_size = 1500, shuffle=True):
        'Initialization'
        self.data = data
        self.sample_size = sample_size
        self.prediction_period = prediction_period
        self.feature_num = feature_num
        self.band_size = band_size
        self.batch_size = batch_size
        self.shuffle = shuffle

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        npy_data = self.data[index*self.batch_size:(index+1)*self.batch_size]
        
        # Generate data
        X, y, s = self.__data_generation(npy_data)

        return X, y, s
    
    def on_epoch_end(self):
        if self.shuffle == True:
            np.random.shuffle(self.data)
    
    def __data_generation(self, data):
        # generate X, Y
        shape = data.shape
        X = np.zeros((shape[0]-self.sample_size, self.sample_size, self.feature_num), dtype=np.float16)
        Y = np.zeros(shape=(shape[0]-self.sample_size, 1), dtype=np.int)
        for i in range(shape[0]-self.sample_size):
            # take the first feature_num columns as features
            X[i] = data[i:i+self.sample_size, 1:self.feature_num+1]
            delta_last = (data[i+self.sample_size-1, 0] - data[i, 0]) / data[i+self.sample_size-1, 0]
            if delta_last < -band_size:
                Y[i] = 0
            elif delta_last > band_size:
                Y[i] = 2
            else:
                Y[i] = 1
        # add the 4th dimension: 1 channel
        X = X.reshape(X.shape[0], self.sample_size, self.feature_num, 1)

        # calculate sample_weights for Y
        sample_weights_y = np.append(Y.flatten(), [0,1,2]) # to ensure exhaustive coverage
        sample_weights_categories = class_weight.compute_class_weight('balanced', classes = np.unique(sample_weights_y), y=sample_weights_y)
        idx = 0
        sample_weights = np.zeros(shape[0]-self.sample_size)
        for y in Y.flatten(): 
            sample_weights[idx] = sample_weights_categories[y]
            idx += 1

        # transform y to categorical arrays
        y_labels = to_categorical(sample_weights_y)[:-3]
                
        return X, y_labels, sample_weights

In [10]:
train_data_generator = DataGenerator(train_data, datasample_period, prediction_period, feature_columns, band_size)

In [11]:
def generate_data(directory, sample_size, prediction_period, feature_num, band_size, v=False):
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith((".npy")):
                # data = np.load(os.path.join(subdir, file))[:temp]
                data = np.load(os.path.join(subdir, file))

                # filter files that are too small
                if v: print('processing', file)
                if data.shape[0] < sample_size:
                    if v: print(file, 'skipped due to insufficient size')
                    continue
                    
                # generate X, Y
                shape = data.shape
                X = np.zeros((shape[0]-sample_size, sample_size, feature_num), dtype=np.float16)
                Y = np.zeros(shape=(shape[0]-sample_size, 1), dtype=np.int)
                for i in range(shape[0]-sample_size):
                    # take the first feature_num columns as features
                    X[i] = data[i:i+sample_size, 1:feature_num+1]
                    delta_last = (data[i+sample_size-1, 0] - data[i, 0]) / data[i+sample_size-1, 0]
                    if delta_last < -band_size:
                        Y[i] = 0
                    elif delta_last > band_size:
                        Y[i] = 2
                    else:
                        Y[i] = 1
                # add the 4th dimension: 1 channel
                X = X.reshape(X.shape[0], sample_size, feature_num, 1)
                
                # calculate sample_weights for Y
                sample_weights_y = np.append(Y.flatten(), [0,1,2]) # to ensure exhaustive coverage
                sample_weights_categories = class_weight.compute_class_weight('balanced', classes = np.unique(sample_weights_y), y=sample_weights_y)
                idx = 0
                sample_weights = np.zeros(shape[0]-sample_size)
                for y in Y.flatten(): 
                    sample_weights[idx] = sample_weights_categories[y]
                    idx += 1

                # transform y to categorical arrays
                y_labels = to_categorical(sample_weights_y)[:-3]
                
                # stratify into batches
                total_size = np.prod(X.shape)
                batch_size = math.floor(total_size / capacity_const) + 1
                batches_x = np.array_split(X, batch_size)
                batches_y = np.array_split(y_labels, batch_size)
                batches_sample_weights = np.array_split(sample_weights, batch_size)
                for n in range(batch_size):
                    yield batches_x[n], batches_y[n], batches_sample_weights[n]

In [12]:
#batch = generate_data(train_dir, datasample_period, prediction_period, feature_columns, band_size, True)

In [13]:
# calculate how many steps per epoch required
steps_per_epoch = 3135 #steps per epoch identified 2021-03-15
#steps_per_epoch = sum(1 for dummy in batch)
print(steps_per_epoch, 'steps per epoch identified')

3135 steps per epoch identified


In [14]:
# dataset = next(batch)
# dataset

In [15]:
# dataset[0].shape

In [16]:
# dataset[1].shape

In [17]:
# dataset[2].shape

In [18]:
def make_model(datasample_period, feature_columns):
    input_tensor = Input(shape=(datasample_period,feature_columns,1))

    # convolutional filter is (1,2) with stride of (1,2)
    layer_x = layers.Conv2D(16, (1,2), strides=(1,2))(input_tensor)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

    layer_x = layers.Conv2D(16, (1,2), strides=(1,2))(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

    layer_x = layers.Conv2D(16, (1,10))(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

    # Inception Module
    tower_1 = layers.Conv2D(32, (1,1), padding='same')(layer_x)
    tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)
    tower_1 = layers.Conv2D(32, (3,1), padding='same')(tower_1)
    tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)

    tower_2 = layers.Conv2D(32, (1,1), padding='same')(layer_x)
    tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)
    tower_2 = layers.Conv2D(32, (5,1), padding='same')(tower_2)
    tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)  

    tower_3 = layers.MaxPooling2D((3,1), padding='same', strides=(1,1))(layer_x)
    tower_3 = layers.Conv2D(32, (1,1), padding='same')(tower_3)
    tower_3 = layers.LeakyReLU(alpha=0.01)(tower_3)

    layer_x = layers.concatenate([tower_1, tower_2, tower_3], axis=-1)

    # concatenate features of tower_1, tower_2, tower_3
    layer_x = layers.Reshape((datasample_period,96))(layer_x)

    # 64 LSTM units
    layer_x = LSTM(64)(layer_x)
    # The last output layer uses a softmax activation function
    # output = layers.Dense(3, activation='softmax')(layer_x)
    x = layers.Dense(3)(x)
    output = layers.Activation('softmax', dtype='float32')(x)
    output = layers.Activation('linear', dtype='float32')(output)
    
    model = Model(input_tensor, output)

    model.summary()
    model.initial_epoch = 0
    opt = tf.keras.optimizers.Adam(lr=0.01, epsilon=1) # learning rate and epsilon are the same as paper DeepLOB
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

In [19]:
def make_or_restore_model(datasample_period,feature_columns):
    # Either restore the latest model, or create a fresh one
    # if there is no checkpoint available.
    checkpoints = [checkpoint_dir + '/' + name
                   for name in os.listdir(checkpoint_dir)]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print('Restoring from', latest_checkpoint)
        model = load_model(latest_checkpoint)
        print(latest_checkpoint)
        model.initial_epoch = int(latest_checkpoint[latest_checkpoint.index("epoch=")+6:])
        return model
    print('Creating a new model')
    return make_model(datasample_period,feature_columns)

In [20]:
model = make_or_restore_model(datasample_period,feature_columns)

Restoring from ./models/checkpoint/ckpt-loss=0.51-epoch=0103
./models/checkpoint/ckpt-loss=0.51-epoch=0103


In [21]:
callbacks = [
    # This callback saves a SavedModel every 100 batches.
    # We include the training loss in the folder name.
    keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_dir + '/ckpt-loss={loss:.2f}-epoch={epoch:04d}/',
        save_freq='epoch', save_weights_only=False)
]

In [22]:
train_generator = generate_data(train_dir, datasample_period, prediction_period, feature_columns, band_size)
t1 = itertools.cycle(train_generator)
validate_generator = generate_data(validate_dir, datasample_period, prediction_period, feature_columns, band_size)
v1 = itertools.cycle(validate_generator)

In [23]:
initial_epoch = model.initial_epoch
# model.fit(train_generator, epochs=10, steps_per_epoch=n_train_files, validation_data=validate_generator, validation_steps=n_validate_files, callbacks=callbacks, initial_epoch=initial_epoch)
# model.fit(t1, epochs=200, steps_per_epoch=steps_per_epoch, validation_data=v1, validation_steps=n_validate_files, callbacks=callbacks, initial_epoch=initial_epoch)
model.fit(train_data_generator, epochs=200, callbacks=callbacks, initial_epoch=initial_epoch)

Epoch 104/200
2710/2710 [==============================] - 824s 301ms/step - loss: 0.4800 - accuracy: 0.7905


INFO:tensorflow:Assets written to: ./models/checkpoint/ckpt-loss=0.48-epoch=0104\assets


INFO:tensorflow:Assets written to: ./models/checkpoint/ckpt-loss=0.48-epoch=0104\assets


Epoch 105/200
  10/2710 [..............................] - ETA: 13:23 - loss: 1.8610 - accuracy: 0.3683

KeyboardInterrupt: 